In [1]:
import cv2
import numpy as np
from skimage.feature import ORB, match_descriptors
from skimage.transform import ProjectiveTransform, warp
from skimage.color import rgb2gray
from skimage import img_as_ubyte

# Charger les deux images que vous souhaitez recalibrer
image1 = cv2.imread('20230323_191223_BRS_L_UP_11.369_00.png.jpg')
image2 = cv2.imread('20230406_060421_BRS_L_UP_11.369_00.png.jpg')

# Convertir les images en niveaux de gris
gray_image1 = rgb2gray(image1)
gray_image2 = rgb2gray(image2)

# Détection de points d'intérêt et extraction de descripteurs avec scikit-image
orb = ORB(n_keypoints=1000)
orb.detect_and_extract(gray_image1)
keypoints1 = orb.keypoints
descriptors1 = orb.descriptors

orb.detect_and_extract(gray_image2)
keypoints2 = orb.keypoints
descriptors2 = orb.descriptors

# Mise en correspondance des descripteurs
matches = match_descriptors(descriptors1, descriptors2, cross_check=True)

# Extraire les points correspondants des deux images
src_pts = keypoints1[matches[:, 0]][:, ::-1]  # Inverser les colonnes pour (x, y)
dst_pts = keypoints2[matches[:, 1]][:, ::-1]

# Estimation de la transformation (homographie)
model = ProjectiveTransform()
model.estimate(src_pts, dst_pts)

# Appliquer la transformation à l'image 1 pour la recalibrer sur l'image 2
aligned_image = warp(image1, model.inverse, output_shape=(image2.shape[0], image2.shape[1]))

# Convertir l'image alignée en entiers non signés 8 bits (pour l'affichage)
aligned_image = img_as_ubyte(aligned_image)

# Concaténer les deux images recalées horizontalement
result_image = np.hstack((image2, aligned_image))

# Afficher les images originales et recalées côte à côte
cv2.imshow('Images originales et recalées', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[ WARN:0@0.739] global loadsave.cpp:248 findDecoder imread_('20230323_191223_BRS_L_UP_11.369_00.png.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.740] global loadsave.cpp:248 findDecoder imread_('20230406_060421_BRS_L_UP_11.369_00.png.jpg'): can't open/read file: check file path/integrity


IndexError: tuple index out of range